 ## Первый раздел.
 ### Импорт библиотек, подключение CUDA devices

In [ ]:
!pip install numba
!pip install py-cpuinfo

import numba
import math
import time
from  datetime  import  datetime 
import numpy as np
from numba import cuda, jit, float64, njit, prange, vectorize
import pandas as pd
import cpuinfo 

### Проверяем, какое оборудование у нас подключенно.

In [ ]:
print('----------------------------------------------------------------------')
print('CPU, выделенный на сервере Colaboratory:', cpuinfo.get_cpu_info()['brand_raw'])
print('GPU, выделенный на сервере Colaboratory:', cuda.get_current_device())
print('----------------------------------------------------------------------')
numba.cuda.detect()

----------------------------------------------------------------------
CPU, выделенный на сервере Colaboratory: Intel(R) Xeon(R) CPU @ 2.30GHz
GPU, выделенный на сервере Colaboratory: <CUDA device 0 'b'Tesla K80''>
----------------------------------------------------------------------
Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

### Второй раздел.
Обьявляем матрицы, а так же необходимо создать собственное ядро, указав количество блоков на сетку и количество потоков на блок. Продукт двух даст общее количество запущенных потоков. Создание экземпляра ядра выполняется путем взятия скомпилированной функции ядра и ее индексации. с набором целых чисел.

In [ ]:
size = [100, 300, 700, 1100]                         
A_0 = np.random.random((size[0], size[0]))    
B_0 = np.random.random((size[0], size[0])) 
C_0 = np.zeros((size[0], size[0]))        
G_0 = np.zeros((size[0], size[0]))

A_1 = np.random.randn(size[1], size[1])
B_1 = np.random.randn(size[1], size[1])
C_1 = np.zeros((size[1], size[1]))
G_1 = np.zeros((size[1], size[1]))

A_2 = np.random.randn(size[2], size[2])
B_2 = np.random.randn(size[2], size[2])
C_2 = np.zeros((size[2], size[2]))
G_2 = np.zeros((size[2], size[2]))

A_3 = np.random.randn(size[3], size[3])
B_3 = np.random.randn(size[3], size[3])
C_3 = np.zeros((size[3], size[3]))
G_3 = np.zeros((size[3], size[3]))

Arr_A = [A_0, A_1, A_2, A_3] 
Arr_B = [B_0, B_1, B_2, B_3]  
ArrC_C = [C_0, C_1, C_2, C_3]
ArrG_C = [G_0, G_1, G_2, G_3]
Arr_timeCPU = []
Arr_timeGPU = []
CPU_time =[]
GPU_time =[]

### Функция перемножения матриц

In [ ]:
def CPU_matmul(A, B, C):
    '''умножение матриц на процессоре'''


    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            summation = 0
            for k in range(A.shape[1]):
                summation += A[i, k] * B[k, j]
            C[i, j] = summation
    return C

Производим перемножение на CPU:

In [ ]:
for i in range(4):
  start = time.time()
  ArrC_C[i] = CPU_matmul(Arr_A[i], Arr_B[i], ArrC_C[i]) 
  Arr_timeCPU.append(time.time() - start)
  print('MatMul №',i+1,'on CPU:', round(Arr_timeCPU[i],2), "second")

MatMul № 1 on CPU: 0.65 second
MatMul № 2 on CPU: 17.23 second
MatMul № 3 on CPU: 222.06 second
MatMul № 4 on CPU: 836.7 second


# Третий раздел.
## Вычисления на GPU:

Функция вычисления на GPU

In [ ]:
@cuda.jit
def GPU_matmul(A, B, C):
    '''умножение матриц на графическом процессоре, наивный метод,
     с использованием глобальной памяти устройства.'''

    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        summation = 0
        for k in range(A.shape[1]):
            summation += A[i, k] * B[k, j]
        C[i, j] = summation

In [ ]:
def host_naive(A, B, C):
  '''Копируем на девайс матрицы, задаем размер блока, 
  количество блоков и вызваем функцию перемножения.'''
  d_A = cuda.to_device(A)
  d_B = cuda.to_device(B)
  d_C = cuda.device_array(C.shape, np.float64)
  
  TPB = 32

  threadsperblock = (TPB, TPB)
  blockspergrid_x = math.ceil(A.shape[0]/threadsperblock[0])
  blockspergrid_y = math.ceil(B.shape[1]/threadsperblock[1])
  blockspergrid = (blockspergrid_x, blockspergrid_y)

  GPU_matmul[blockspergrid, threadsperblock](d_A, d_B, d_C)
  return d_C.copy_to_host()

cuda.select_device(0) #выбор устройства GPU

<weakproxy at 0x7fba639a0410 to Device at 0x7fba639a4f10>

Производим перемножение на GPU:

In [ ]:
for i in range(4):
  start = time.time()
  ArrG_C[i] = host_naive(Arr_A[i], Arr_B[i], ArrG_C[i]) 
  Arr_timeGPU.append(time.time() - start)
  print('MatMul №',i+1,'on GPU:', round(Arr_timeGPU[i],2), "second") 

MatMul № 1 on GPU: 0.23 second
MatMul № 2 on GPU: 0.02 second
MatMul № 3 on GPU: 0.12 second
MatMul № 4 on GPU: 0.39 second


# Четвертый раздел.
##Сравнения результатов вычислений:

In [ ]:
import datetime
num = 0
for i in range(4):
  if np.allclose(ArrC_C[i], ArrG_C[i]):
    num +=1

GPU_time = []
for i in range(4):
  CPU_time.append(str(datetime.timedelta(seconds=round(Arr_timeCPU[i]))))
  GPU_time.append(str(round(Arr_timeGPU[i], 2)))

if num == 4:
  print("Расчеты - верны! Матрицы идентичны!")
  
else:
    print("Расчеты не верны!")

Расчеты - верны! Матрицы идентичны!


In [ ]:
timing_df = pd.DataFrame({'Device':['CPU', 'GPU', 'CPU' ,'GPU', 'CPU', 'GPU', 'CPU' ,'GPU'],
                          'Time(h:m:s)':[CPU_time[0], GPU_time[0], CPU_time[1], GPU_time[1], CPU_time[2], GPU_time[2], CPU_time[3], GPU_time[3]],
                          'Matrix_size':[f"{size[0]} x {size[0]}", f"{size[0]} x {size[0]}", f"{size[1]} x {size[1]}", f"{size[1]} x {size[1]}",
                                         f"{size[2]} x {size[2]}", f"{size[2]} x {size[2]}", f"{size[3]} x {size[3]}", f"{size[3]} x {size[3]}"]})
timing_df 

,Device,Time(h:m:s),Matrix_size
0,CPU,0:00:01,100 x 100
1,GPU,0.23,100 x 100
2,CPU,0:00:17,300 x 300
3,GPU,0.02,300 x 300
4,CPU,0:03:42,700 x 700
5,GPU,0.12,700 x 700
6,CPU,0:13:57,1100 x 1100
7,GPU,0.39,1100 x 1100
